In [ ]:
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord, Angle
from IPython.display import Image
from astropy.table import Table
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path

#gammapy_dependencies
from gammapy.data import DataStore, Observation
from gammapy.maps import Map, MapAxis, RegionGeom, WcsGeom
from gammapy.datasets import Datasets, SpectrumDataset
from gammapy.makers import (
    ReflectedRegionsBackgroundMaker,
    SafeMaskMaker,
    SpectrumDatasetMaker,
    WobbleRegionsFinder,)
from gammapy.visualization import plot_spectrum_datasets_off_regions
from gammapy.modeling.models import (
    SpectralModel,
    ExpCutoffPowerLawSpectralModel,
    LogParabolaSpectralModel,
    SkyModel,
    PowerLawSpectralModel,
    SPECTRAL_MODEL_REGISTRY,
)
from gammapy.modeling import Fit
from gammapy.estimators import FluxPointsEstimator
from gammapy.astro.darkmatter import DarkMatterAnnihilationSpectralModel
from gammapy.stats import wstat, WStatCountsStatistic
from regions import PointSkyRegion, CircleSkyRegion
import logging
from matplotlib.lines import Line2D


In [ ]:
# Path to your DL3 files
data_store = DataStore.from_dir("/home/abhishek/Desktop/GalCent/trial/Point_old")

In [ ]:
#Extract the observations from the DL3 files

observations = data_store.get_observations(required_irf="point-like") 

# change the required_irf to "full-enclosure"/"["aeff","edisp","psf","rad_max"]"

In [ ]:
# this function clubs all the observations correspoinding to the same wobble pointings, the possible wobbles are 1,2,3,4

def obs_in_wobble(wobble):    
    if wobble == 1:
        obs_ids = []
        i = 1
        for _obs in observations:
            if _obs.pointing_radec.separation(observations[0].pointing_radec).to_value("deg") <= 0.4:
                #print(_obs.obs_id)
                obs_ids.append(_obs)
                i = i+1
        n=i
    if wobble == 2:
        obs_ids = []
        i = 1
        for _obs in observations:
            if _obs.pointing_radec.separation(observations[2].pointing_radec).to_value("deg") <= 0.4:
                #print(_obs.obs_id)
                obs_ids.append(_obs)
                i = i+1
        n=i
    if wobble == 3:
        obs_ids = []
        i = 1
        for _obs in observations:
            if _obs.pointing_radec.separation(observations[4].pointing_radec).to_value("deg") <= 0.4:
                #print(_obs.obs_id)
                obs_ids.append(_obs)
                i = i+1
        n=i
    if wobble == 4:
        obs_ids = []
        i = 1
        for _obs in observations:
            if _obs.pointing_radec.separation(observations[7].pointing_radec).to_value("deg") <= 0.4:
                #print(_obs.obs_id)
                obs_ids.append(_obs)
                i = i+1
        n=i      
    return obs_ids

In [ ]:
# The function below creates the legend for the counts map

def create_fake_legend(ax, source_name, excluded_source_name):
    """Make a fake legend representing the ON and OFF regions"""
    markersize = 14
    on = Line2D(
        [0],
        [0],
        markeredgecolor="crimson",
        markerfacecolor="none",
        markeredgewidth=1.5,
        marker="o",
        markersize=markersize,
        ls="",
        label="on region",
    )
    off = Line2D(
        [0],
        [0],
        markeredgecolor="green",
        markerfacecolor="none",
        markeredgewidth=1.5,
        marker="o",
        markersize=markersize,
        ls="",
        label="off regions",
    )
    pointing = Line2D(
        [0],
        [0],
        markeredgecolor="goldenrod",
        markerfacecolor="none",
        markeredgewidth=1.5,
        marker="+",
        markersize=(markersize - 2),
        ls="",
        label="pointing",
    )
    source = Line2D(
        [0],
        [0],
        markeredgecolor="crimson",
        markerfacecolor="none",
        markeredgewidth=1.5,
        marker="*",
        markersize=(markersize - 2),
        ls="",
        label=source_name,
    )
    excluded_source = Line2D(
        [0],
        [0],
        markeredgecolor="k",
        markerfacecolor="none",
        markeredgewidth=1.5,
        marker="*",
        markersize=(markersize - 2),
        ls="",
        label=excluded_source_name,
    )
    excluded_region = Line2D(
        [0],
        [0],
        markeredgecolor="k",
        markerfacecolor="none",
        markeredgewidth=1.5,
        marker="o",
        markersize=markersize,
        ls="",
        label="excluded region",
    )
    ax.legend(
        handles=[on, off, pointing, source, excluded_source, excluded_region],
        loc="best",
    )

In [ ]:
# Below is the function to plot on and off regions for one set of observations corresponding to one wobble pointing

def plot_stacked_obs_on_off_regions(
    obs_list,
    skymap_geom,
    source_coords,
    exclusion_region_center,
    exclusion_region_radius,
    source_name,
    excluded_source_name,
    show=True,
):
    rad_max = 0.1   ##np.max(observation.rad_max.data)

    # plotting starts here
    fig = plt.figure()
    counts = Map.from_geom(skymap_geom)
    for obs in obs_list:
        counts.fill_events(obs.events)
    wcs = counts.geom.wcs
    # plot the countmap
    ax = counts.plot(cmap="viridis", add_cbar=True)
    wobble_radius = obs_list[0].pointing_radec.separation(source_coords).to_value("deg")
    pointing = PointSkyRegion(obs_list[0].pointing_radec)
    pointing.to_pixel(wcs).plot(ax=ax, color="goldenrod", marker="+", markersize=12)
    wobble_circle = CircleSkyRegion(center=obs_list[0].pointing_radec, radius=Angle(f"{wobble_radius} deg"))
    wobble_circle.to_pixel(wcs).plot(ax=ax, edgecolor="goldenrod", ls="--", linewidth=2)

    # plot the ON region, create a new one with an actual extension
    on_region_circle = CircleSkyRegion(center=source_coords, radius=Angle("0.253 deg"))   #Angle(f"{rad_max} deg"))
    on_region_circle.to_pixel(wcs).plot(ax=ax, edgecolor="crimson", linewidth=2)

    # plot the source positions and the exclusion mask
    #PointSkyRegion(source_coords).to_pixel(wcs).plot(ax=ax, color="crimson", marker=".", markersize=12)
    PointSkyRegion(exclusion_region_center).to_pixel(wcs).plot(ax=ax, color="k", marker="*", markersize=12)
    exclusion_region = CircleSkyRegion(center=exclusion_region_center, radius=exclusion_region_radius)
    exclusion_region.to_pixel(wcs).plot(ax=ax, edgecolor="k", ls="-", linewidth=2)

    # create the OFF regions, this is the same process performed by the BackgroundMaker in the dataset
    region_finder = WobbleRegionsFinder(n_off_regions=3)
    # find the OFF regions centers
    off_regions, wcs = region_finder.run(region=PointSkyRegion(source_coords), center=obs_list[0].pointing_radec)

    # plot the OFF regions
    for off_region in off_regions:
        off_region_circle = CircleSkyRegion(center=off_region.center, radius=Angle(f"{rad_max} deg"))
        off_region_circle.to_pixel(ax.wcs).plot(ax=ax, edgecolor="blue", linewidth=2)

    create_fake_legend(ax=ax, source_name=source_name, excluded_source_name=excluded_source_name)
    ax.set_title(f"for one of the wobble with 3 off regions")
    if show:
        plt.show()

In [ ]:
# Define the source of interest regions and eclusion regions if any
#For example I am taking the coordinates for galactic center and pulsar wind nebulae G0.9+0.1 as exclusion region


gc_coordinates = SkyCoord.from_name("Galactic Center", frame="icrs")
g0901 = SkyCoord(frame="icrs", ra=266.825 * u.deg, dec=-28.15 * u.deg)
on_center = PointSkyRegion(gc_coordinates)
exclusion_region = CircleSkyRegion(center=g0901, radius=0.3 * u.deg)

skymap_geom = WcsGeom.create(skydir=gc_coordinates, binsz=0.05, width="5 deg", frame="icrs", proj="TAN")
#exclusion_mask_g0901 = ~skymap_geom.region_mask([exclusion_region])

In [ ]:
# get one set of observations for one of the wobble pointing

obs_1 = obs_in_wobble(1)

In [ ]:
# Plot the on_off counts map

plot_stacked_obs_on_off_regions(
    obs_list=obs_1, 
    skymap_geom=skymap_geom, 
    source_coords=gc_coordinates,
    exclusion_region_center=exclusion_region.center,
    exclusion_region_radius=Angle("0.3 deg"),
    source_name="GC",
    excluded_source_name="G0.9+0.1"
)